## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Wed Jul 23 09:07:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install Ultralytics


In [3]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.2/112.6 GB disk)


## Download data from aws

In [4]:
!pip install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [5]:
from google.colab import userdata

# grant access to secrets bu checking them
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ["AWS_DEFAULT_REGION"] = userdata.get('AWS_DEFAULT_REGION')

In [18]:
!cd /{HOME}
!ls

content  rtdetr-l.pt  runs


In [6]:
!aws s3 cp s3://bucket-sail-0/data/ ./data/ --recursive

download: s3://bucket-sail-0/data/split_summary.json to content/datasets/split_summary.json
download: s3://bucket-sail-0/data/classes.txt to content/datasets/classes.txt            
download: s3://bucket-sail-0/data/data.yaml to content/datasets/data.yaml                
download: s3://bucket-sail-0/data/notes.json to content/datasets/notes.json              
download: s3://bucket-sail-0/data/train/images/frame_1VCrw7QVQog_45.00s.jpg to content/datasets/train/images/frame_1VCrw7QVQog_45.00s.jpg
download: s3://bucket-sail-0/data/train/images/frame_1VCrw7QVQog_16.00s.jpg to content/datasets/train/images/frame_1VCrw7QVQog_16.00s.jpg
download: s3://bucket-sail-0/data/train/images/frame_1VCrw7QVQog_15.00s.jpg to content/datasets/train/images/frame_1VCrw7QVQog_15.00s.jpg
download: s3://bucket-sail-0/data/train/images/frame_1VCrw7QVQog_25.00s.jpg to content/datasets/train/images/frame_1VCrw7QVQog_25.00s.jpg
download: s3://bucket-sail-0/data/train/images/frame_1VCrw7QVQog_13.00s.jpg to content

In [30]:

from ultralytics import RTDETR

from IPython.display import display, Image
model = RTDETR('rtdetr-l.pt')
from pathlib import Path

model.train(data='/content/data/data.yaml',
            epochs=30,
            imgsz=1088,
            batch=6,
            workers=4,
            project='runs/train',
            name='rtdetr_exp',
            augment=True,
            degrees=45,)

Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/data.yaml, degrees=45, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rtdetr_exp, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tr

train: Scanning /content/data/train_oversampled/labels.cache... 871 images, 0 backgrounds, 0 corrupt: 100%|██████████| 871/871 [00:00<?, ?it/s]

train: /content/data/train_oversampled/images/frame_llU5yYlyjPI_169.20s.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 562.5±40.7 MB/s, size: 255.7 KB)


val: Scanning /content/data/val/labels.cache... 105 images, 21 backgrounds, 0 corrupt: 100%|██████████| 105/105 [00:00<?, ?it/s]


...

  0%|          | 0/436 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
        1/1      2.19G      1.615       4.52      0.435          6        640: 100%|██████████| 436/436 [02:42<00:00,  2.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:03<00:00,  6.99it/s]


                   all        105        344      0.154      0.173       0.08     0.0242

1 epochs completed in 0.048 hours.
Optimizer stripped from runs/train/rtdetr_exp/weights/last.pt, 66.1MB
Optimizer stripped from runs/train/rtdetr_exp/weights/best.pt, 66.1MB

Validating runs/train/rtdetr_exp/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
rt-detr-l summary: 302 layers, 31,989,905 parameters, 0 gradients, 103.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:04<00:00,  6.24it/s]


                   all        105        344      0.155      0.173     0.0801     0.0241
       pennon_attached         65        213      0.139     0.0704     0.0585     0.0188
       pennon_detached         42        102     0.0936      0.206     0.0511     0.0153
          pennon_leech         21         29      0.231      0.241      0.131     0.0382
Speed: 0.3ms preprocess, 34.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/train/rtdetr_exp


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aa0100e76d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [32]:
!aws s3 sync runs/train s3://bucket-sail-1/runs/train

upload: runs/train/rtdetr_exp/results.csv to s3://bucket-sail-1/runs/train/rtdetr_exp/results.csv
upload: runs/train/rtdetr_exp/BoxPR_curve.png to s3://bucket-sail-1/runs/train/rtdetr_exp/BoxPR_curve.png
upload: runs/train/rtdetr_exp/args.yaml to s3://bucket-sail-1/runs/train/rtdetr_exp/args.yaml
upload: runs/train/rtdetr_exp/train_batch0.jpg to s3://bucket-sail-1/runs/train/rtdetr_exp/train_batch0.jpg
upload: runs/train/rtdetr_exp/BoxR_curve.png to s3://bucket-sail-1/runs/train/rtdetr_exp/BoxR_curve.png
upload: runs/train/rtdetr_exp/labels.jpg to s3://bucket-sail-1/runs/train/rtdetr_exp/labels.jpg
upload: runs/train/rtdetr_exp/confusion_matrix.png to s3://bucket-sail-1/runs/train/rtdetr_exp/confusion_matrix.png
upload: runs/train/rtdetr_exp/BoxF1_curve.png to s3://bucket-sail-1/runs/train/rtdetr_exp/BoxF1_curve.png
upload: runs/train/rtdetr_exp/train_batch2.jpg to s3://bucket-sail-1/runs/train/rtdetr_exp/train_batch2.jpg
upload: runs/train/rtdetr_exp/BoxP_curve.png to s3://bucket-sail